In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gc; gc.enable()
import pickle
from tqdm import tqdm
import warnings ; warnings.filterwarnings('ignore')
import os
from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions

In [2]:
folder = 'final oofs/'
os.listdir(folder)

['all_mean_enc_lgb_oof_test_pred.csv',
 'all_mean_enc_lgb_oof_val_pred.csv',
 'all_mean_enc_user_feat2_lgb_oof_test_pred.csv',
 'all_mean_enc_user_feat2_lgb_oof_val_pred.csv',
 'all_mean_enc_user_feat_lgb_oof_test_pred.csv',
 'all_mean_enc_user_feat_lgb_oof_val_pred.csv',
 'alpha_0001_oof_test_pred.csv',
 'alpha_0001_oof_val_pred.csv',
 'alpha_10_oof_test_pred.csv',
 'alpha_10_oof_val_pred.csv',
 'alpha_160_oof_test_pred.csv',
 'alpha_160_oof_val_pred.csv',
 'alpha_320_oof_test_pred.csv',
 'alpha_320_oof_val_pred.csv',
 'baseline_xgb_oof_test_pred.csv',
 'baseline_xgb_oof_val_pred.csv',
 'catboost1_without_text_oof_test_pred',
 'catboost1_without_text_oof_test_pred.csv',
 'catboost1_without_text_oof_val_pred',
 'catboost1_without_text_oof_val_pred.csv',
 'catboost_oof_test_pred.csv',
 'catboost_oof_val_pred.csv',
 'cat_interact_lgb_oof_test_pred.csv',
 'cat_interact_lgb_oof_val_pred.csv',
 'cgb_with_categorical_oof_test_pred.csv',
 'cgb_with_categorical_oof_val_pred.csv',
 'cls05_lgb_o

In [3]:
bi_interact_statics = [
    'xentropy_add_lotsof_image_features_lgb',
    'lgb411_tune',
    'plants_lgb',
    'plants_with_img_meta_nima_fm_geo_active_lgb',
    'simple_feature_lgb',
    'cat_interact_lgb',
    'mixed_features_text_proprocessing_lgb',
    'marcus_lgb',
    'lgb411_dart_tune',
    'poisson_small_lr_lgb',
    'img_meta_xgb',
    'baseline_xgb',
    'catboost1_without_text',
    'cgb_with_categorical',
    'mcl_cgb',
    'selftrained_bigru_conv1d_rnn',
    'pretrained_bigru_cv1d_rnn',
    'selftrained_bigru_conv1d_merged_with_image_adv_rnn',
    'text_lgb',
    'text_rg',
    'mlp',
    'alpha_0001',
    'ridge_new_411',
    'rf_411_7000_leaves'
]

tri_interact_statics = [
    'xentropy_add_lotsof_image_features_lgb',
    'lgb411_tune',
    'simple_feature_lgb',
    'lgb411_dart_tune',
    'poisson_small_lr_lgb',
    'img_meta_xgb',
    'baseline_xgb',
    'catboost1_without_text',
    'selftrained_bigru_conv1d_rnn',
    'selftrained_bigru_conv1d_merged_with_image_adv_rnn',
    'text_lgb',
    'text_rg',
    'mlp',
    'alpha_0001',
    'ridge_new_411',
    'rf_411_7000_leaves'
]

configs = [
    {
        'name': 'lgb',
        'prefixs': [
            'xentropy_add_lotsof_image_features_lgb',
            'lgb411_tune',
            'plants_lgb', #411
            'plants_with_img_meta_nima_fm_geo_active_lgb',
            'plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb',
            #'xentropy_small_lr_lgb', #lgb411_tune
            'xentropy_small_lr_cat_lgb',
            'simple_feature_lgb', #411
            'all_mean_enc_lgb', #411
            'all_mean_enc_user_feat_lgb', #411
            'all_mean_enc_user_feat2_lgb', #411
            'cat_interact_lgb', #411
            'mean_enc_lgb', #411
            'marcus_lgb', #o411
            'fused_text_lgb', #o411
            'mixed_features_text_proprocessing_lgb', #o411,
            'select_dense_features_lgb', #411
            'select_sparse_features_lgb', #411
        ],
    },
    {
        'name': 'lgb_dart',
        'prefixs': [
            'lgb411_dart_tune',
        ],
    },
    {
        'name': 'lgb_goss',
        'prefixs': [
            'lgb411_goss_tune',
            'lgb411_goss_tune2',
        ],
    },
    {
        'name': 'lgb_pois',
        'prefixs': [
            'poisson_lgb', #o411
            'poisson_small_lr_lgb',
        ]
    },
    {
        'name': 'xgb_lg',
        'prefixs': [
            'small_features_v5_xgb', #o411
            'small_features_v4_xgb', #o411
            'nima_features_xgb', #o411
            'img_meta_xgb', #o411
            'img_meta_nima_xgb', #o411
            'xgb_new',
        ]
    },
    {
        'name': 'xgb_dw',
        'prefixs': [
            'baseline_xgb', #o411
        ]
    },
    {
        'name': 'xgb_ranking',
        'prefixs': [
            'ranking_xgb', #o411
        ]
    },
    {
        'name': 'catboost',
        'prefixs': [
            'catboost', #411
            'catboost1_without_text',
            'mcl_cgb',
            'cgb_with_categorical',
        ]
    },
    {
        'name': 'rnn',
        'prefixs': [
            'pretrained_bigru_cv1d_rnn', #411
            'pretrained_bigru_attention_rnn', #411
            'pretrained_2gru_rnn', #411
            'selftrained_bigru_conv1d_rnn', #411
            'selftrained_bigru_conv1d_merged_with_image_rnn',
            'selftrained_bigru_conv1d_merged_with_image_adv_rnn',
        ]
    },
    {
        'name': 'text',
        'prefixs': [
            'text_lgb',#411
            'text_cwb_rg',#411
            'text_fm', #411
            'text_rg', #411
        ]
    },
    {
        'name': 'mlp',
        'prefixs': [
            'mlp',#411
        ]
    },
    {
        'name': 'random_forest',
        'prefixs': [
            'rf',#411
            'rf_411_7000_leaves',
        ]
    },
    {
        'name': 'ridge',
        'prefixs': [
            'alpha_0001',#411
            'alpha_160',#411
            'alpha_10',#411
            'alpha_320',#411
            'ridge_new_411',
        ]
    },
    {
        'name': 'logistic_regression',
        'prefixs': [
            'lr_l1_05',#411
            'lr_l1_1',#411
            'lr_l2_01',#411
            'lr_l2_1',#411
        ]
    },
    {
        'name': 'lgb_cls05',
        'prefixs': [
            'cls05_lgb',
        ]
    },
    {
        'name': 'lgb_cls0',
        'prefixs': [
            'cls0_lgb',
        ]
    },
    {
        'name': 'multiclass',
        'prefixs': [
            'multiclass_lgb',
        ]
    },
    {
        'name': 'multiclass3',
        'prefixs': [
            'multiclass3_lgb',
        ]
    }
]

In [4]:
train = pd.DataFrame()
test = pd.DataFrame()

In [5]:
from scipy.stats import hmean
from scipy.stats.mstats import gmean

def get_clipped_values(a):
    return np.clip(a, 1e-15, 1.)

basic_columns = []
for config in configs:
    columns = []
    print('Processing group: ', config['name'])
    for prefix in config['prefixs']:
        train_f = folder + prefix + '_oof_val_pred.csv'
        test_f = folder + prefix + '_oof_test_pred.csv'
        
        train_df = pd.read_csv(train_f)
        test_df = pd.read_csv(test_f)
        
        if config['name'] == 'multiclass' or config['name'] == 'multiclass3':
            original_cols = train_df.columns.tolist()
            
            for c in original_cols:
                col = prefix+c
                print('Add ', col)
                
                train.loc[:,col] = train_df[c]
                test.loc[:,col] = test_df[c]
        else:
            original_col = train_df.columns.tolist()[0]
            col = prefix
            print('Add ', col)
            columns.append(col)
            basic_columns.append(col)
            
            train.loc[:,col] = train_df[original_col]
            test.loc[:,col] = test_df[original_col]
        
        del train_df, test_df; gc.collect()
    
    # apply feature engineering on intra-group columns
    if len(columns) < 2:
        continue

    for df in [train, test]:    
        # within sample group
        df.loc[:, config['name']+'_mean'] = df[columns].mean(axis=1)
        df.loc[:, config['name']+'_med'] = df[columns].median(axis=1)
        df.loc[:, config['name']+'_max'] = df[columns].max(axis=1)
        df.loc[:, config['name']+'_min'] = df[columns].min(axis=1)
        df.loc[:, config['name']+'_std'] = df[columns].std(axis=1)


# apply feature engineering on inter_group columns
for df in [train, test]:    
    df.loc[:, 'bi_inter_group_mean']  = df[bi_interact_statics].mean(axis=1)
    df.loc[:, 'bi_inter_group_med']   = df[bi_interact_statics].median(axis=1)
    df.loc[:, 'bi_inter_group_max']   = df[bi_interact_statics].max(axis=1)
    df.loc[:, 'bi_inter_group_min']   = df[bi_interact_statics].min(axis=1)
    df.loc[:, 'bi_inter_group_std']   = df[bi_interact_statics].std(axis=1)
    df.loc[:, 'tri_inter_group_mean'] = df[tri_interact_statics].mean(axis=1)
    df.loc[:, 'tri_inter_group_med']  = df[tri_interact_statics].median(axis=1)
    df.loc[:, 'tri_inter_group_max']  = df[tri_interact_statics].max(axis=1)
    df.loc[:, 'tri_inter_group_min']  = df[tri_interact_statics].min(axis=1)
    df.loc[:, 'tri_inter_group_std']  = df[tri_interact_statics].std(axis=1)
    
    '''
    col_len = len(bi_interact_statics)
    for i in range(col_len-1):
        for j in range(i+1, col_len):
            cols = [bi_interact_statics[i], bi_interact_statics[j]]
            feat_name = cols[0]+'_'+cols[1]+'_inter'
            print('Add ', feat_name, ' statistcs')        
            df.loc[:, feat_name+'_mean'] = df[cols].mean(axis=1)
            df.loc[:, feat_name+'_gmean'] = gmean(get_clipped_values(df[cols].values), axis=1)
            #df.loc[:, feat_name+'_hmean'] = hmean(get_clipped_values(df[cols].values), axis=1)
    
    
    col_len = len(tri_interact_statics)
    for i in range(col_len-2):
        for j in range(i+1, col_len-1):
            for k in range(j+1, col_len):
                cols = [tri_interact_statics[i], tri_interact_statics[j], tri_interact_statics[k]]
                feat_name = cols[0]+'_'+cols[1]+'_'+cols[2]+'_inter'  
                print('Add ', feat_name, ' statistcs')                    
                df.loc[:, feat_name+'_mean'] = df[cols].mean(axis=1)
                df.loc[:, feat_name+'_gmean'] = gmean(get_clipped_values(df[cols].values), axis=1)
                #df.loc[:, feat_name+'_hmean'] = hmean(get_clipped_values(df[cols].values), axis=1)
                df.loc[:, feat_name+'_med'] = df[cols].median(axis=1)
                df.loc[:, feat_name+'_std'] = df[cols].std(axis=1)             
    '''            
    #df = df.astype(np.float32)

Processing group:  lgb
Add  xentropy_add_lotsof_image_features_lgb
Add  lgb411_tune
Add  plants_lgb
Add  plants_with_img_meta_nima_fm_geo_active_lgb
Add  plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb
Add  xentropy_small_lr_cat_lgb
Add  simple_feature_lgb
Add  all_mean_enc_lgb
Add  all_mean_enc_user_feat_lgb
Add  all_mean_enc_user_feat2_lgb
Add  cat_interact_lgb
Add  mean_enc_lgb
Add  marcus_lgb
Add  fused_text_lgb
Add  mixed_features_text_proprocessing_lgb
Add  select_dense_features_lgb
Add  select_sparse_features_lgb
Processing group:  lgb_dart
Add  lgb411_dart_tune
Processing group:  lgb_goss
Add  lgb411_goss_tune
Add  lgb411_goss_tune2
Processing group:  lgb_pois
Add  poisson_lgb
Add  poisson_small_lr_lgb
Processing group:  xgb_lg
Add  small_features_v5_xgb
Add  small_features_v4_xgb
Add  nima_features_xgb
Add  img_meta_xgb
Add  img_meta_nima_xgb
Add  xgb_new
Processing group:  xgb_dw
Add  baseline_xgb
Processing group:  xgb_ranking
Add  ranking_xgb
Processing group:  cat

In [6]:
train.head(3)

,xentropy_add_lotsof_image_features_lgb,lgb411_tune,plants_lgb,plants_with_img_meta_nima_fm_geo_active_lgb,plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb,xentropy_small_lr_cat_lgb,simple_feature_lgb,all_mean_enc_lgb,all_mean_enc_user_feat_lgb,all_mean_enc_user_feat2_lgb,...,bi_inter_group_mean,bi_inter_group_med,bi_inter_group_max,bi_inter_group_min,bi_inter_group_std,tri_inter_group_mean,tri_inter_group_med,tri_inter_group_max,tri_inter_group_min,tri_inter_group_std
0,0.075277,0.062362,0.053896,0.071850,0.059415,0.047786,0.068429,0.054375,0.054969,0.052336,...,0.051283,0.052869,0.080676,0.004070,0.020358,0.046161,0.042857,0.080676,0.004070,0.021428
1,0.039158,0.041640,0.057651,0.057721,0.033101,0.062043,0.040434,0.015163,0.015783,0.017107,...,0.033417,0.035638,0.061023,-0.047206,0.022739,0.029550,0.035638,0.061023,-0.047206,0.024273
2,0.047349,0.031383,0.040794,0.048324,0.039097,0.052663,0.036796,0.036656,0.034745,0.035856,...,0.039608,0.038222,0.061303,0.019574,0.009547,0.038380,0.036989,0.056999,0.019574,0.009321


In [7]:
pd.options.display.max_rows=100
pd.options.display.max_columns=100
train[basic_columns].corr()

,xentropy_add_lotsof_image_features_lgb,lgb411_tune,plants_lgb,plants_with_img_meta_nima_fm_geo_active_lgb,plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb,xentropy_small_lr_cat_lgb,simple_feature_lgb,all_mean_enc_lgb,all_mean_enc_user_feat_lgb,all_mean_enc_user_feat2_lgb,cat_interact_lgb,mean_enc_lgb,marcus_lgb,fused_text_lgb,mixed_features_text_proprocessing_lgb,select_dense_features_lgb,select_sparse_features_lgb,lgb411_dart_tune,lgb411_goss_tune,lgb411_goss_tune2,poisson_lgb,poisson_small_lr_lgb,small_features_v5_xgb,small_features_v4_xgb,nima_features_xgb,img_meta_xgb,img_meta_nima_xgb,xgb_new,baseline_xgb,ranking_xgb,catboost,catboost1_without_text,mcl_cgb,cgb_with_categorical,pretrained_bigru_cv1d_rnn,pretrained_bigru_attention_rnn,pretrained_2gru_rnn,selftrained_bigru_conv1d_rnn,selftrained_bigru_conv1d_merged_with_image_rnn,selftrained_bigru_conv1d_merged_with_image_adv_rnn,text_lgb,text_cwb_rg,text_fm,text_rg,mlp,rf,rf_411_7000_leaves,alpha_0001,alpha_160,alpha_10,alpha_320,ridge_new_411,lr_l1_05,lr_l1_1,lr_l2_01,lr_l2_1,cls05_lgb,cls0_lgb
xentropy_add_lotsof_image_features_lgb,1.000000,0.987515,0.962282,0.971707,0.979859,0.963546,0.902607,0.904278,0.904221,0.896114,0.898852,0.903478,0.915897,0.912013,0.913005,0.902741,0.908337,0.976410,0.986276,0.988411,0.979855,0.983149,0.926844,0.922592,0.931074,0.936005,0.938847,0.980519,0.971096,0.765870,0.893589,0.893589,0.972385,0.970223,0.892459,0.891782,0.889217,0.891598,0.900249,0.915566,0.822430,0.685411,0.821406,0.823256,0.864836,0.921385,0.935732,0.811904,0.846438,0.827642,0.850100,0.872428,0.811401,0.799280,0.821263,0.793754,0.938616,0.785823
lgb411_tune,0.987515,1.000000,0.967159,0.976990,0.986189,0.967743,0.905456,0.907395,0.907480,0.899326,0.902366,0.906453,0.919712,0.914794,0.915861,0.906655,0.912361,0.982080,0.994217,0.996717,0.985545,0.988986,0.931473,0.927126,0.935678,0.940697,0.943550,0.987608,0.977573,0.769518,0.896485,0.896485,0.976032,0.974701,0.895204,0.894450,0.891942,0.894094,0.896064,0.912031,0.823135,0.685161,0.821971,0.823708,0.866779,0.922428,0.937665,0.813002,0.847543,0.828779,0.851161,0.873289,0.813043,0.800982,0.822777,0.795435,0.945198,0.788349
plants_lgb,0.962282,0.967159,1.000000,0.968862,0.963568,0.951730,0.911633,0.912456,0.912490,0.904745,0.907654,0.912110,0.914469,0.921208,0.921965,0.896150,0.902178,0.966236,0.968448,0.967875,0.963319,0.966409,0.931010,0.926905,0.930084,0.925493,0.924579,0.962428,0.955797,0.765113,0.890247,0.890247,0.969725,0.964663,0.897465,0.896696,0.894225,0.897245,0.892095,0.903323,0.835321,0.695687,0.834781,0.836749,0.874204,0.931922,0.942972,0.824736,0.860072,0.840884,0.863767,0.880408,0.823825,0.811323,0.833793,0.805571,0.919283,0.781393
plants_with_img_meta_nima_fm_geo_active_lgb,0.971707,0.976990,0.968862,1.000000,0.973449,0.957623,0.904752,0.906086,0.906017,0.898062,0.900886,0.905318,0.915583,0.914908,0.915626,0.901881,0.907734,0.972245,0.977728,0.977729,0.972814,0.976023,0.924954,0.920816,0.928658,0.931048,0.933453,0.971450,0.964709,0.768467,0.894999,0.894999,0.976016,0.972276,0.891990,0.891134,0.888897,0.891522,0.891001,0.904735,0.828427,0.690939,0.827950,0.830028,0.867792,0.927867,0.941243,0.818223,0.853142,0.834198,0.856787,0.879468,0.816884,0.804517,0.826794,0.798866,0.931634,0.782935
plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb,0.979859,0.986189,0.963568,0.973449,1.000000,0.962748,0.900203,0.901806,0.901888,0.893871,0.896375,0.900974,0.912759,0.910385,0.911364,0.899385,0.905112,0.976626,0.985737,0.986809,0.982530,0.985805,0.925209,0.920967,0.929248,0.933461,0.936151,0.979641,0.972157,0.766018,0.891253,0.891253,0.971611,0.968867,0.890066,0.889372,0.886792,0.889311,0.890608,0.906435,0.821424,0.684999,0.820779,0.822768,0.862835,0.921629,0.935721,0.811344,0.845992,0.827171,0.849634,0.872109,0.811378,0.799215,0.821176,0.793686,0.940242,0.785084
xentropy_small_lr_cat_lgb,0.963546,0.967743,0.951730,0.957623,0.962748,1.000000,0.914189,0.917104,0.916845,0.907965,0.911571,0.916972,0.932491,0.925647,0

In [8]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 132 columns):
xentropy_add_lotsof_image_features_lgb                      float64
lgb411_tune                                                 float64
plants_lgb                                                  float64
plants_with_img_meta_nima_fm_geo_active_lgb                 float64
plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb    float64
xentropy_small_lr_cat_lgb                                   float64
simple_feature_lgb                                          float64
all_mean_enc_lgb                                            float64
all_mean_enc_user_feat_lgb                                  float64
all_mean_enc_user_feat2_lgb                                 float64
cat_interact_lgb                                            float64
mean_enc_lgb                                                float64
marcus_lgb                                                  float64
fu

In [9]:
train.shape, test.shape

((1503424, 132), (508438, 132))

In [10]:
print(train.isnull().sum().max()) 
print(test.isnull().sum().max())

0
0


In [11]:
for col in tqdm(train.columns):
    train[col] = train[col].astype(np.float32)
    test[col] = test[col].astype(np.float32)

100%|████████████████████████████████████████████████████████████████████████████| 132/132 [00:57<00:00,  2.31it/s]


In [12]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 132 columns):
xentropy_add_lotsof_image_features_lgb                      float32
lgb411_tune                                                 float32
plants_lgb                                                  float32
plants_with_img_meta_nima_fm_geo_active_lgb                 float32
plants_with_img_meta_nima_fm_geo_active_obj_xentropy_lgb    float32
xentropy_small_lr_cat_lgb                                   float32
simple_feature_lgb                                          float32
all_mean_enc_lgb                                            float32
all_mean_enc_user_feat_lgb                                  float32
all_mean_enc_user_feat2_lgb                                 float32
cat_interact_lgb                                            float32
mean_enc_lgb                                                float32
marcus_lgb                                                  float32
fu

In [13]:
with open('meta_train_411_simple.pickle', 'wb') as handle:
    pickle.dump(train, handle)
    
with open('meta_test_411_simple.pickle', 'wb') as handle:
    pickle.dump(test, handle)

## Meta Model Tuning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gc; gc.enable()
import pickle
from tqdm import tqdm
import warnings ; warnings.filterwarnings('ignore')
import os
from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions, clip_rmse

In [2]:
SEED=411
train_y = pd.read_csv("regression_target.csv").deal_probability.values

In [3]:
with open('meta_train.pickle', 'rb') as handle:
    train = pickle.load(handle)
    
with open('meta_test.pickle', 'rb') as handle:
    test = pickle.load(handle)

In [4]:
train.shape

(1503424, 684)

### Lightgbm-gbdt

In [4]:
ml = model_loader(model_type='lgb')

In [6]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'min_split_gain': [.0, .1, .2, .3, .4]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'min_split_gain': 0.0} train loss: 0.207717, valid loss:0.208867, loss_diff:0.001150
{'min_split_gain': 0.0} train loss: 0.207552, valid loss:0.209566, loss_diff:0.002014
{'min_split_gain': 0.0} train loss: 0.207818, valid loss:0.208491, loss_diff:0.000673
{'min_split_gain': 0.0} train loss: 0.207751, valid loss:0.208752, loss_diff:0.001001
{'min_split_gain': 0.0} train loss: 0.207555, valid loss:0.209447, loss_diff:0.001891
=================>{'min_split_gain': 0.0} loss:0.209025
{'min_split_gain': 0.1} train loss: 0.207720, valid loss:0.208860, loss_diff:0.001140
{'min_split_gain': 0.1} train loss: 0.207555, valid loss:0.209577, loss_diff:0.002021
{'min_split_gain': 0.1} train loss: 0.207806, valid loss:0.208466, loss_diff:0.000660
{'min_split_gain': 0.1} train loss: 0.207734, valid loss:0.208763, loss_diff:0.001029
{'min_split_gain': 0.1} train loss: 0.207557, valid loss:0.209453, loss_diff:0.001896
=================>{'min_split_gain': 0.1} loss:0.209024
{'min_split_gain': 0.2} trai

KeyboardInterrupt: 

In [14]:
'''
best
{'min_split_gain': 0.0} train loss: 0.207978, valid loss:0.208974, loss_diff:0.000996
{'min_split_gain': 0.0} train loss: 0.207808, valid loss:0.209675, loss_diff:0.001867
{'min_split_gain': 0.0} train loss: 0.208046, valid loss:0.208600, loss_diff:0.000554
{'min_split_gain': 0.0} train loss: 0.207981, valid loss:0.208823, loss_diff:0.000843
{'min_split_gain': 0.0} train loss: 0.207785, valid loss:0.209536, loss_diff:0.001751
=================>{'min_split_gain': 0.0} loss:0.209122


best 
{'min_split_gain': 0.1} train loss: 0.208348, valid loss:0.209236, loss_diff:0.000887
{'min_split_gain': 0.1} train loss: 0.208166, valid loss:0.209885, loss_diff:0.001718
{'min_split_gain': 0.1} train loss: 0.208426, valid loss:0.208832, loss_diff:0.000405
{'min_split_gain': 0.1} train loss: 0.208379, valid loss:0.208949, loss_diff:0.000570
{'min_split_gain': 0.1} train loss: 0.208165, valid loss:0.209724, loss_diff:0.001559
=================>{'min_split_gain': 0.1} loss:0.209325
'''

"\nbest\n{'min_split_gain': 0.0} train loss: 0.207978, valid loss:0.208974, loss_diff:0.000996\n{'min_split_gain': 0.0} train loss: 0.207808, valid loss:0.209675, loss_diff:0.001867\n{'min_split_gain': 0.0} train loss: 0.208046, valid loss:0.208600, loss_diff:0.000554\n{'min_split_gain': 0.0} train loss: 0.207981, valid loss:0.208823, loss_diff:0.000843\n{'min_split_gain': 0.0} train loss: 0.207785, valid loss:0.209536, loss_diff:0.001751\n=================>{'min_split_gain': 0.0} loss:0.209122\n\n\nbest \n{'min_split_gain': 0.1} train loss: 0.208348, valid loss:0.209236, loss_diff:0.000887\n{'min_split_gain': 0.1} train loss: 0.208166, valid loss:0.209885, loss_diff:0.001718\n{'min_split_gain': 0.1} train loss: 0.208426, valid loss:0.208832, loss_diff:0.000405\n{'min_split_gain': 0.1} train loss: 0.208379, valid loss:0.208949, loss_diff:0.000570\n{'min_split_gain': 0.1} train loss: 0.208165, valid loss:0.209724, loss_diff:0.001559\n=================>{'min_split_gain': 0.1} loss:0.2093

In [5]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None
# 1.0: 0.209025
try_params = {
    'colsample_bytree':[i/10.0 for i in range(7,10)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'colsample_bytree': 0.7} train loss: 0.207786, valid loss:0.208842, loss_diff:0.001056
{'colsample_bytree': 0.7} train loss: 0.207601, valid loss:0.209568, loss_diff:0.001967
{'colsample_bytree': 0.7} train loss: 0.207868, valid loss:0.208521, loss_diff:0.000653
{'colsample_bytree': 0.7} train loss: 0.207817, valid loss:0.208786, loss_diff:0.000969
{'colsample_bytree': 0.7} train loss: 0.207611, valid loss:0.209465, loss_diff:0.001854
=================>{'colsample_bytree': 0.7} loss:0.209036
{'colsample_bytree': 0.8} train loss: 0.207777, valid loss:0.208878, loss_diff:0.001101
{'colsample_bytree': 0.8} train loss: 0.207576, valid loss:0.209557, loss_diff:0.001982
{'colsample_bytree': 0.8} train loss: 0.207803, valid loss:0.208481, loss_diff:0.000678
{'colsample_bytree': 0.8} train loss: 0.207753, valid loss:0.208760, loss_diff:0.001007
{'colsample_bytree': 0.8} train loss: 0.207593, valid loss:0.209485, loss_diff:0.001892
=================>{'colsample_bytree': 0.8} loss:0.209032
{'co

,param,val_loss_mean,val_loss_std
0,{'colsample_bytree': 0.7},0.209036,0.000408
1,{'colsample_bytree': 0.8},0.209032,0.000420
2,{'colsample_bytree': 0.9},0.209010,0.000409


In [6]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':.9, 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None

# subsample 1.0 => cv: 0.209122  new: 0.209010
try_params = {
    'subsample':[i/10.0 for i in range(7,10)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'subsample': 0.7} train loss: 0.207751, valid loss:0.208873, loss_diff:0.001122
{'subsample': 0.7} train loss: 0.207616, valid loss:0.209654, loss_diff:0.002038
{'subsample': 0.7} train loss: 0.207830, valid loss:0.208497, loss_diff:0.000667
{'subsample': 0.7} train loss: 0.207769, valid loss:0.208789, loss_diff:0.001020
{'subsample': 0.7} train loss: 0.207594, valid loss:0.209526, loss_diff:0.001932
=================>{'subsample': 0.7} loss:0.209068
{'subsample': 0.8} train loss: 0.207735, valid loss:0.208928, loss_diff:0.001193
{'subsample': 0.8} train loss: 0.207583, valid loss:0.209582, loss_diff:0.001998
{'subsample': 0.8} train loss: 0.207815, valid loss:0.208513, loss_diff:0.000698
{'subsample': 0.8} train loss: 0.207759, valid loss:0.208773, loss_diff:0.001014
{'subsample': 0.8} train loss: 0.207580, valid loss:0.209517, loss_diff:0.001937
=================>{'subsample': 0.8} loss:0.209062
{'subsample': 0.9} train loss: 0.207729, valid loss:0.208872, loss_diff:0.001143
{'subsa

,param,val_loss_mean,val_loss_std
0,{'subsample': 0.7},0.209068,0.000446
1,{'subsample': 0.8},0.209062,0.000420
2,{'subsample': 0.9},0.209045,0.000417


In [7]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':.9, 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None

# 2.0: cv 0.209052306045, new: 0.209010
try_params = {
    'reg_alpha':[1.0, 1.5, 2.5, 3.0]
} 

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_alpha': 1.0} train loss: 0.207717, valid loss:0.208923, loss_diff:0.001206
{'reg_alpha': 1.0} train loss: 0.207541, valid loss:0.209613, loss_diff:0.002072
{'reg_alpha': 1.0} train loss: 0.207771, valid loss:0.208505, loss_diff:0.000734
{'reg_alpha': 1.0} train loss: 0.207712, valid loss:0.208783, loss_diff:0.001072
{'reg_alpha': 1.0} train loss: 0.207519, valid loss:0.209447, loss_diff:0.001928
=================>{'reg_alpha': 1.0} loss:0.209054
{'reg_alpha': 1.5} train loss: 0.207708, valid loss:0.208872, loss_diff:0.001164
{'reg_alpha': 1.5} train loss: 0.207541, valid loss:0.209611, loss_diff:0.002070
{'reg_alpha': 1.5} train loss: 0.207790, valid loss:0.208473, loss_diff:0.000684
{'reg_alpha': 1.5} train loss: 0.207738, valid loss:0.208749, loss_diff:0.001011
{'reg_alpha': 1.5} train loss: 0.207546, valid loss:0.209477, loss_diff:0.001931
=================>{'reg_alpha': 1.5} loss:0.209036
{'reg_alpha': 2.5} train loss: 0.207741, valid loss:0.208882, loss_diff:0.001141
{'reg_a

,param,val_loss_mean,val_loss_std
0,{'reg_alpha': 1.0},0.209054,0.000414
1,{'reg_alpha': 1.5},0.209036,0.000436
2,{'reg_alpha': 2.5},0.209026,0.000413
3,{'reg_alpha': 3.0},0.209016,0.000417


In [8]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':.9, 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None

try_params = {
    'reg_alpha':[3.5, 4.0, 4.5]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_alpha': 3.5} train loss: 0.207790, valid loss:0.208848, loss_diff:0.001058
{'reg_alpha': 3.5} train loss: 0.207620, valid loss:0.209600, loss_diff:0.001980
{'reg_alpha': 3.5} train loss: 0.207859, valid loss:0.208476, loss_diff:0.000617
{'reg_alpha': 3.5} train loss: 0.207780, valid loss:0.208757, loss_diff:0.000977
{'reg_alpha': 3.5} train loss: 0.207595, valid loss:0.209449, loss_diff:0.001854
=================>{'reg_alpha': 3.5} loss:0.209026
{'reg_alpha': 4.0} train loss: 0.207794, valid loss:0.208861, loss_diff:0.001067
{'reg_alpha': 4.0} train loss: 0.207608, valid loss:0.209546, loss_diff:0.001938
{'reg_alpha': 4.0} train loss: 0.207833, valid loss:0.208444, loss_diff:0.000611
{'reg_alpha': 4.0} train loss: 0.207805, valid loss:0.208773, loss_diff:0.000968
{'reg_alpha': 4.0} train loss: 0.207614, valid loss:0.209456, loss_diff:0.001842
=================>{'reg_alpha': 4.0} loss:0.209016
{'reg_alpha': 4.5} train loss: 0.207790, valid loss:0.208847, loss_diff:0.001057
{'reg_a

,param,val_loss_mean,val_loss_std
0,{'reg_alpha': 3.5},0.209026,0.000428
1,{'reg_alpha': 4.0},0.209016,0.000421
2,{'reg_alpha': 4.5},0.209015,0.000423


In [9]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':.9, 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None

try_params = {
    'reg_alpha':[5.0, 5.5, 6.]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_alpha': 5.0} train loss: 0.207808, valid loss:0.208846, loss_diff:0.001039
{'reg_alpha': 5.0} train loss: 0.207638, valid loss:0.209516, loss_diff:0.001878
{'reg_alpha': 5.0} train loss: 0.207882, valid loss:0.208444, loss_diff:0.000563
{'reg_alpha': 5.0} train loss: 0.207833, valid loss:0.208766, loss_diff:0.000933
{'reg_alpha': 5.0} train loss: 0.207638, valid loss:0.209462, loss_diff:0.001824
=================>{'reg_alpha': 5.0} loss:0.209007
{'reg_alpha': 5.5} train loss: 0.207810, valid loss:0.208814, loss_diff:0.001005
{'reg_alpha': 5.5} train loss: 0.207641, valid loss:0.209524, loss_diff:0.001883
{'reg_alpha': 5.5} train loss: 0.207892, valid loss:0.208459, loss_diff:0.000567
{'reg_alpha': 5.5} train loss: 0.207840, valid loss:0.208743, loss_diff:0.000903
{'reg_alpha': 5.5} train loss: 0.207646, valid loss:0.209441, loss_diff:0.001795
=================>{'reg_alpha': 5.5} loss:0.208996
{'reg_alpha': 6.0} train loss: 0.207814, valid loss:0.208857, loss_diff:0.001042
{'reg_a

,param,val_loss_mean,val_loss_std
0,{'reg_alpha': 5.0},0.209007,0.000416
1,{'reg_alpha': 5.5},0.208996,0.000415
2,{'reg_alpha': 6.0},0.208997,0.000424


In [10]:
default_params = {
    'boosting_type':'gbdt', 
    'objective': 'xentropy',
    'num_leaves':31, 
    'max_depth':8, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':.9, 
    'reg_alpha':5.5, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 4
}

fit_param = None

# reg_lambda 0.0: 0.209121440793  new: Best params: {'reg_alpha': 5.5} 	best loss: 0.208996368394
try_params = {
    'reg_lambda':[i/10.0 for i in range(1,4)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_lambda': 0.1} train loss: 0.207796, valid loss:0.208817, loss_diff:0.001021
{'reg_lambda': 0.1} train loss: 0.207639, valid loss:0.209526, loss_diff:0.001887
{'reg_lambda': 0.1} train loss: 0.207892, valid loss:0.208459, loss_diff:0.000567
{'reg_lambda': 0.1} train loss: 0.207847, valid loss:0.208756, loss_diff:0.000909
{'reg_lambda': 0.1} train loss: 0.207640, valid loss:0.209425, loss_diff:0.001785
=================>{'reg_lambda': 0.1} loss:0.208997
{'reg_lambda': 0.2} train loss: 0.207809, valid loss:0.208841, loss_diff:0.001032
{'reg_lambda': 0.2} train loss: 0.207640, valid loss:0.209526, loss_diff:0.001887
{'reg_lambda': 0.2} train loss: 0.207892, valid loss:0.208459, loss_diff:0.000567
{'reg_lambda': 0.2} train loss: 0.207847, valid loss:0.208756, loss_diff:0.000909
{'reg_lambda': 0.2} train loss: 0.207645, valid loss:0.209449, loss_diff:0.001803
=================>{'reg_lambda': 0.2} loss:0.209006
{'reg_lambda': 0.3} train loss: 0.207809, valid loss:0.208842, loss_diff:0.0

KeyboardInterrupt: 

In [ ]:
default_params = {
    'boosting_type':'gbdt', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.02, 
    'n_estimators':3000, 
    'min_split_gain':0.1, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':.7,  
    'colsample_bytree':1., 
    'reg_alpha':1.5, 
    'reg_lambda':0.7, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = {
    'early_stopping_rounds': 50,
    'verbose': 100,
    'eval_metric': 'rmse'
}

_, ret_test, _ = get_oof_predictions(train, train_y, test, ml, 
                                     default_params, seed=19, fit_params=fit_param, use_eval_set=True)

In [ ]:
test_df = pd.read_csv("test.csv", usecols=['item_id'])
pd.DataFrame(np.clip(ret_test,0,1), 
             index=test_df.item_id,
             columns=['deal_probability']).to_csv('lgb_meta_no_bagging_exclude_knn.csv')

### Lightgbm-dart

In [6]:
default_params = {
    'boosting_type':'dart', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.0, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'min_split_gain': [.0, .1, .2, .3, .4]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'min_split_gain': 0.0} train loss: 0.213927, valid loss:0.214264, loss_diff:0.000337
{'min_split_gain': 0.0} train loss: 0.213748, valid loss:0.215104, loss_diff:0.001356
{'min_split_gain': 0.0} train loss: 0.214077, valid loss:0.213627, loss_diff:-0.000450
{'min_split_gain': 0.0} train loss: 0.214023, valid loss:0.213778, loss_diff:-0.000245
{'min_split_gain': 0.0} train loss: 0.213804, valid loss:0.214721, loss_diff:0.000917
=================>{'min_split_gain': 0.0} loss:0.214299
{'min_split_gain': 0.1} train loss: 0.213920, valid loss:0.214252, loss_diff:0.000331
{'min_split_gain': 0.1} train loss: 0.213755, valid loss:0.215107, loss_diff:0.001352
{'min_split_gain': 0.1} train loss: 0.214055, valid loss:0.213606, loss_diff:-0.000449
{'min_split_gain': 0.1} train loss: 0.214021, valid loss:0.213776, loss_diff:-0.000245
{'min_split_gain': 0.1} train loss: 0.213790, valid loss:0.214729, loss_diff:0.000938
=================>{'min_split_gain': 0.1} loss:0.214294
{'min_split_gain': 0.2} 

KeyboardInterrupt: 

In [23]:
default_params = {
    'boosting_type':'dart', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.3, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'colsample_bytree':[i/10.0 for i in range(6,11)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'colsample_bytree': 0.6} train loss: 0.212777, valid loss:0.212907, loss_diff:0.000130
{'colsample_bytree': 0.6} train loss: 0.212682, valid loss:0.213379, loss_diff:0.000696
{'colsample_bytree': 0.6} train loss: 0.212666, valid loss:0.213448, loss_diff:0.000782
{'colsample_bytree': 0.6} train loss: 0.212721, valid loss:0.213123, loss_diff:0.000403
{'colsample_bytree': 0.6} train loss: 0.212778, valid loss:0.212892, loss_diff:0.000114
=================>{'colsample_bytree': 0.6} loss:0.213150
{'colsample_bytree': 0.7} train loss: 0.212761, valid loss:0.212934, loss_diff:0.000172
{'colsample_bytree': 0.7} train loss: 0.212644, valid loss:0.213358, loss_diff:0.000714
{'colsample_bytree': 0.7} train loss: 0.212643, valid loss:0.213420, loss_diff:0.000777
{'colsample_bytree': 0.7} train loss: 0.212692, valid loss:0.213109, loss_diff:0.000417
{'colsample_bytree': 0.7} train loss: 0.212771, valid loss:0.212889, loss_diff:0.000118
=================>{'colsample_bytree': 0.7} loss:0.213142
{'co

,param,val_loss_mean,val_loss_std
0,{'colsample_bytree': 0.6},0.213150,0.000231
1,{'colsample_bytree': 0.7},0.213142,0.000216
2,{'colsample_bytree': 0.8},0.213123,0.000223
3,{'colsample_bytree': 0.9},0.213140,0.000212
4,{'colsample_bytree': 1.0},0.213110,0.000217


In [24]:
default_params = {
    'boosting_type':'dart', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.3, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'subsample':[i/10.0 for i in range(6,11)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'subsample': 0.6} train loss: 0.212709, valid loss:0.212853, loss_diff:0.000144
{'subsample': 0.6} train loss: 0.212645, valid loss:0.213329, loss_diff:0.000684
{'subsample': 0.6} train loss: 0.212630, valid loss:0.213384, loss_diff:0.000754
{'subsample': 0.6} train loss: 0.212655, valid loss:0.213066, loss_diff:0.000410
{'subsample': 0.6} train loss: 0.212758, valid loss:0.212853, loss_diff:0.000095
=================>{'subsample': 0.6} loss:0.213097
{'subsample': 0.7} train loss: 0.212711, valid loss:0.212867, loss_diff:0.000157
{'subsample': 0.7} train loss: 0.212632, valid loss:0.213330, loss_diff:0.000698
{'subsample': 0.7} train loss: 0.212620, valid loss:0.213375, loss_diff:0.000755
{'subsample': 0.7} train loss: 0.212664, valid loss:0.213054, loss_diff:0.000390
{'subsample': 0.7} train loss: 0.212737, valid loss:0.212844, loss_diff:0.000107
=================>{'subsample': 0.7} loss:0.213094
{'subsample': 0.8} train loss: 0.212731, valid loss:0.212885, loss_diff:0.000154
{'subsa

,param,val_loss_mean,val_loss_std
0,{'subsample': 0.6},0.213097,0.000226
1,{'subsample': 0.7},0.213094,0.000223
2,{'subsample': 0.8},0.213108,0.000221
3,{'subsample': 0.9},0.213126,0.000219
4,{'subsample': 1.0},0.213110,0.000217


In [25]:
default_params = {
    'boosting_type':'dart', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.3, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':.7,  
    'colsample_bytree':1., 
    'reg_alpha':2.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'reg_alpha':[1.0, 1.5, 2.0, 2.5, 3.0]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_alpha': 1.0} train loss: 0.212702, valid loss:0.212864, loss_diff:0.000162
{'reg_alpha': 1.0} train loss: 0.212614, valid loss:0.213305, loss_diff:0.000691
{'reg_alpha': 1.0} train loss: 0.212582, valid loss:0.213371, loss_diff:0.000788
{'reg_alpha': 1.0} train loss: 0.212675, valid loss:0.213072, loss_diff:0.000397
{'reg_alpha': 1.0} train loss: 0.212702, valid loss:0.212821, loss_diff:0.000118
=================>{'reg_alpha': 1.0} loss:0.213086
{'reg_alpha': 1.5} train loss: 0.212732, valid loss:0.212871, loss_diff:0.000139
{'reg_alpha': 1.5} train loss: 0.212631, valid loss:0.213312, loss_diff:0.000681
{'reg_alpha': 1.5} train loss: 0.212605, valid loss:0.213374, loss_diff:0.000770
{'reg_alpha': 1.5} train loss: 0.212669, valid loss:0.213057, loss_diff:0.000388
{'reg_alpha': 1.5} train loss: 0.212743, valid loss:0.212854, loss_diff:0.000111
=================>{'reg_alpha': 1.5} loss:0.213093
{'reg_alpha': 2.0} train loss: 0.212711, valid loss:0.212867, loss_diff:0.000157
{'reg_a

,param,val_loss_mean,val_loss_std
0,{'reg_alpha': 1.0},0.213086,0.000223
1,{'reg_alpha': 1.5},0.213093,0.000217
2,{'reg_alpha': 2.0},0.213094,0.000223
3,{'reg_alpha': 2.5},0.213104,0.000216
4,{'reg_alpha': 3.0},0.213110,0.000222


In [26]:
default_params = {
    'boosting_type':'dart', 
    'num_leaves':31, 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'min_split_gain':0.3, 
    'min_child_weight':0.001, 
    'min_child_samples':20, 
    'subsample':.7,  
    'colsample_bytree':1., 
    'reg_alpha':1.0, 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'reg_lambda':[i/10.0 for i in range(0,11,1)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'reg_lambda': 0.0} train loss: 0.212702, valid loss:0.212864, loss_diff:0.000162
{'reg_lambda': 0.0} train loss: 0.212614, valid loss:0.213305, loss_diff:0.000691
{'reg_lambda': 0.0} train loss: 0.212582, valid loss:0.213371, loss_diff:0.000788
{'reg_lambda': 0.0} train loss: 0.212675, valid loss:0.213072, loss_diff:0.000397
{'reg_lambda': 0.0} train loss: 0.212702, valid loss:0.212821, loss_diff:0.000118
=================>{'reg_lambda': 0.0} loss:0.213086
{'reg_lambda': 0.1} train loss: 0.212705, valid loss:0.212876, loss_diff:0.000171
{'reg_lambda': 0.1} train loss: 0.212607, valid loss:0.213300, loss_diff:0.000693
{'reg_lambda': 0.1} train loss: 0.212582, valid loss:0.213371, loss_diff:0.000788
{'reg_lambda': 0.1} train loss: 0.212675, valid loss:0.213072, loss_diff:0.000397
{'reg_lambda': 0.1} train loss: 0.212718, valid loss:0.212836, loss_diff:0.000118
=================>{'reg_lambda': 0.1} loss:0.213091
{'reg_lambda': 0.2} train loss: 0.212700, valid loss:0.212859, loss_diff:0.0

,param,val_loss_mean,val_loss_std
0,{'reg_lambda': 0.0},0.213086,0.000223
1,{'reg_lambda': 0.1},0.213091,0.000216
2,{'reg_lambda': 0.2},0.213094,0.000223
3,{'reg_lambda': 0.3},0.213092,0.000222
4,{'reg_lambda': 0.4},0.213097,0.000217
5,{'reg_lambda': 0.5},0.213094,0.000217
6,{'reg_lambda': 0.6},0.213097,0.000216
7,{'reg_lambda': 0.7},0.213096,0.000212
8,{'reg_lambda': 0.8},0.213091,0.000222
9,{'reg_lambda': 0.9},0.213083,0.000214


### XGB-gbdt

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'min_child_weight':[0.001, 0.1, 2, 4, 8]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'gamma':[.0, .1, .2, .3, .4]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'colsample_bytree':[i/10.0 for i in range(6,11)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'subsample':[i/10.0 for i in range(6,11)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'reg_alpha':[1.0, 1.5, 2.0, 2.5, 3.0]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

In [ ]:
ml = model_loader(model_type='xgb')

default_params = {
    'booster':'gbtree', 
    'max_depth':5, 
    'learning_rate':0.1, 
    'n_estimators':100, 
    'gamma':0.0, 
    'min_child_weight':0.001,
    'subsample':1.,  
    'colsample_bytree':1., 
    'reg_alpha':2., 
    'reg_lambda':0.0, 
    'random_state':SEED, 
    'n_jobs': 3
}

fit_param = None

try_params = {
    'reg_lambda':[i/10.0 for i in range(0,11,1)]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

## Rigde

In [4]:
ml = model_loader(model_type='rg')

default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

fit_param = None

try_params = {
    'alpha':[1,]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'alpha': 1} train loss: 0.208715, valid loss:0.208575, loss_diff:-0.000140
{'alpha': 1} train loss: 0.208544, valid loss:0.209267, loss_diff:0.000723
{'alpha': 1} train loss: 0.208741, valid loss:0.208355, loss_diff:-0.000386
{'alpha': 1} train loss: 0.208740, valid loss:0.208457, loss_diff:-0.000283
{'alpha': 1} train loss: 0.208566, valid loss:0.209163, loss_diff:0.000597
=================>{'alpha': 1} loss:0.208764
Best params: {'alpha': 1} 	best loss: 0.208763611475


,param,val_loss_mean,val_loss_std
0,{'alpha': 1},0.208764,0.000377


In [25]:
ml = model_loader(model_type='rg')

default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

fit_param = None

try_params = {
    'alpha':[0.05, 0.1, 0.5]
}

fit_params(train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'alpha': 0.05} train loss: 0.208980, valid loss:0.208786, loss_diff:-0.000193
{'alpha': 0.05} train loss: 0.208807, valid loss:0.209493, loss_diff:0.000686
{'alpha': 0.05} train loss: 0.208991, valid loss:0.208590, loss_diff:-0.000402
{'alpha': 0.05} train loss: 0.209001, valid loss:0.208670, loss_diff:-0.000330
{'alpha': 0.05} train loss: 0.208833, valid loss:0.209363, loss_diff:0.000529
=================>{'alpha': 0.05} loss:0.208980
{'alpha': 0.1} train loss: 0.208980, valid loss:0.208786, loss_diff:-0.000193
{'alpha': 0.1} train loss: 0.208807, valid loss:0.209493, loss_diff:0.000686
{'alpha': 0.1} train loss: 0.208992, valid loss:0.208590, loss_diff:-0.000402
{'alpha': 0.1} train loss: 0.209001, valid loss:0.208670, loss_diff:-0.000330
{'alpha': 0.1} train loss: 0.208833, valid loss:0.209363, loss_diff:0.000530
=================>{'alpha': 0.1} loss:0.208980
{'alpha': 0.5} train loss: 0.208981, valid loss:0.208788, loss_diff:-0.000193
{'alpha': 0.5} train loss: 0.208808, valid los

,param,val_loss_mean,val_loss_std
0,{'alpha': 0.05},0.208980,0.000373
1,{'alpha': 0.1},0.208980,0.000373
2,{'alpha': 0.5},0.208981,0.000372


## Bagging + Ultimate Blending

In [8]:
seeds = [9910019]
config = {
    'lgb_xenp': {
        'ml': model_loader(model_type='lgb'),
        'param': {
            'boosting_type':'gbdt', 
            'objective': 'xentropy',
            'num_leaves':31, 
            'max_depth':8, 
            'learning_rate':0.02, 
            'n_estimators':5000, 
            'min_split_gain':0.0, 
            'min_child_weight':0.001, 
            'min_child_samples':20, 
            'subsample':1.,  
            'colsample_bytree':.9, 
            'reg_alpha':5.5, 
            'reg_lambda':0.0, 
            'random_state':SEED, 
            'n_jobs': 4
        },
        'fit_param': {
            'early_stopping_rounds': 100,
            'verbose': 100,
            'eval_metric': 'rmse'
        }
    },
    'lgb_gbdt':{
        'ml': model_loader(model_type='lgb'),
        'param': {
            'boosting_type':'gbdt', 
            'num_leaves':31, 
            'max_depth':5, 
            'learning_rate':0.02, 
            'n_estimators':5000, 
            'min_split_gain':0.0, 
            'min_child_weight':0.001, 
            'min_child_samples':20, 
            'subsample':1.,  
            'colsample_bytree':1., 
            'reg_alpha':2.5, 
            'reg_lambda':0.1, 
            'random_state':SEED, 
            'n_jobs': 4
        },
        'fit_param': {
            'early_stopping_rounds': 100,
            'verbose': 100,
            'eval_metric': 'rmse'
        }
    },
    'ridge':{
        'ml': model_loader(model_type='rg'),
        'param': {
            'alpha': .5, 
            'fit_intercept': True, 
            'normalize': False, 
            'copy_X': True, 
            'max_iter': None, 
            'tol': 0.001, 
            'solver':'auto', 
            'random_state': SEED
        },
        'fit_param': None
    }
}

In [9]:
results = []

for k,v in config.items():
    print('Training & bagging: ', k)
    res = {
        'val_oof': np.zeros((len(train_y),)),
        'test_oof': np.zeros((test.shape[0],))
    }
    
    for seed in seeds:
        print('Training seed =', seed)
        if 'random_state' in v['param']:
            v['param']['random_state'] = seed
            
        oof_val_pred, oof_test_pred, _ = get_oof_predictions(train, train_y, test, v['ml'], 
                                                          v['param'], seed=SEED, fit_params=v['fit_param'], 
                                                          use_eval_set= v['fit_param'] is not None)
        
        res['val_oof'] += oof_val_pred
        res['test_oof'] += oof_test_pred
    
    res['val_oof'] /= len(seeds)
    res['test_oof'] /= len(seeds)
    
    results.append(res)

Training & bagging:  lgb_xenp
Training seed = 9910019
Training until validation scores don't improve for 100 rounds.
[100]	train's rmse: 0.211218	valid's rmse: 0.211431
[200]	train's rmse: 0.209117	valid's rmse: 0.20949
[300]	train's rmse: 0.208521	valid's rmse: 0.209112
[400]	train's rmse: 0.208123	valid's rmse: 0.20892
[500]	train's rmse: 0.207773	valid's rmse: 0.208795
[600]	train's rmse: 0.20747	valid's rmse: 0.208709
[700]	train's rmse: 0.207195	valid's rmse: 0.20865
[800]	train's rmse: 0.206926	valid's rmse: 0.208597
[900]	train's rmse: 0.206667	valid's rmse: 0.208562
[1000]	train's rmse: 0.206412	valid's rmse: 0.208522
[1100]	train's rmse: 0.206177	valid's rmse: 0.208494
[1200]	train's rmse: 0.205939	valid's rmse: 0.208471
[1300]	train's rmse: 0.205717	valid's rmse: 0.208451
[1400]	train's rmse: 0.20549	valid's rmse: 0.208426
[1500]	train's rmse: 0.205266	valid's rmse: 0.208401
[1600]	train's rmse: 0.205053	valid's rmse: 0.208374
[1700]	train's rmse: 0.204829	valid's rmse: 0.208

Training until validation scores don't improve for 100 rounds.
[100]	train's rmse: 0.21104	valid's rmse: 0.2118
[200]	train's rmse: 0.208938	valid's rmse: 0.210024
[300]	train's rmse: 0.208351	valid's rmse: 0.209699
[400]	train's rmse: 0.20796	valid's rmse: 0.209526
[500]	train's rmse: 0.207608	valid's rmse: 0.209412
[600]	train's rmse: 0.207295	valid's rmse: 0.209342
[700]	train's rmse: 0.207005	valid's rmse: 0.209283
[800]	train's rmse: 0.206744	valid's rmse: 0.209234
[900]	train's rmse: 0.20649	valid's rmse: 0.209198
[1000]	train's rmse: 0.206246	valid's rmse: 0.209173
[1100]	train's rmse: 0.206006	valid's rmse: 0.209143
[1200]	train's rmse: 0.205775	valid's rmse: 0.20912
[1300]	train's rmse: 0.205541	valid's rmse: 0.209101
[1400]	train's rmse: 0.205318	valid's rmse: 0.209081
[1500]	train's rmse: 0.20511	valid's rmse: 0.209071
[1600]	train's rmse: 0.204892	valid's rmse: 0.20905
[1700]	train's rmse: 0.204666	valid's rmse: 0.209032
[1800]	train's rmse: 0.204455	valid's rmse: 0.209013


[100]	train's rmse: 0.21138	valid's rmse: 0.211231
[200]	train's rmse: 0.209303	valid's rmse: 0.209448
[300]	train's rmse: 0.208722	valid's rmse: 0.209122
[400]	train's rmse: 0.208321	valid's rmse: 0.208935
[500]	train's rmse: 0.207959	valid's rmse: 0.208796
[600]	train's rmse: 0.20765	valid's rmse: 0.208712
[700]	train's rmse: 0.207356	valid's rmse: 0.208633
[800]	train's rmse: 0.20709	valid's rmse: 0.208569
[900]	train's rmse: 0.20684	valid's rmse: 0.208517
[1000]	train's rmse: 0.206593	valid's rmse: 0.208468
[1100]	train's rmse: 0.206354	valid's rmse: 0.20843
[1200]	train's rmse: 0.206133	valid's rmse: 0.208408
[1300]	train's rmse: 0.205902	valid's rmse: 0.208372
[1400]	train's rmse: 0.205672	valid's rmse: 0.208341
[1500]	train's rmse: 0.205458	valid's rmse: 0.20832
[1600]	train's rmse: 0.205247	valid's rmse: 0.208303
[1700]	train's rmse: 0.205043	valid's rmse: 0.208287
[1800]	train's rmse: 0.204835	valid's rmse: 0.208273
[1900]	train's rmse: 0.204634	valid's rmse: 0.208252
[2000]	t

In [10]:
results

[{'test_oof': array([ 0.44134881,  0.13818581,  0.18065408, ...,  0.05846316,
          0.46645003,  0.11934828]),
  'val_oof': array([ 0.04269062,  0.02786759,  0.04589495, ...,  0.32469264,
          0.44411177,  0.33554076])},
 {'test_oof': array([ 0.45260529,  0.14440085,  0.18600401, ...,  0.05801202,
          0.45661804,  0.11927631]),
  'val_oof': array([ 0.0431904 ,  0.03153727,  0.04552802, ...,  0.27872548,
          0.51625873,  0.39307008])},
 {'test_oof': array([ 0.41481393,  0.17201893,  0.19537803, ...,  0.05836198,
          0.5179934 ,  0.13196116]),
  'val_oof': array([ 0.04396407,  0.04514281,  0.05289709, ...,  0.26401559,
          0.27708091,  0.24400375])}]

In [11]:
import pickle
pickle.dump(results, open('411_meta_oof_result', 'wb'))

In [18]:
import pickle
pickle.dump(results[1:], open('411_meta_oof_result_ver2', 'wb'))

In [10]:
# 0.208262, depth 5

In [12]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

new_train['f1'] = results[0]['val_oof']
new_train['f2'] = results[1]['val_oof']
new_train['f3'] = results[2]['val_oof']

new_test['f1'] = results[0]['test_oof']
new_test['f2'] = results[1]['test_oof']
new_test['f3'] = results[2]['test_oof']

ml = model_loader(model_type='rg')
default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

fit_param = None

try_params = {
    'alpha':[1,2,4,8]
}

fit_params(new_train, train_y, ml, default_params, try_params, fit_params=fit_param, seed=SEED, use_eval_set=False)

{'alpha': 1} train loss: 0.208249, valid loss:0.208026, loss_diff:-0.000223
{'alpha': 1} train loss: 0.208074, valid loss:0.208732, loss_diff:0.000659
{'alpha': 1} train loss: 0.208334, valid loss:0.207688, loss_diff:-0.000646
{'alpha': 1} train loss: 0.208270, valid loss:0.207946, loss_diff:-0.000323
{'alpha': 1} train loss: 0.208095, valid loss:0.208643, loss_diff:0.000548
=================>{'alpha': 1} loss:0.208207
{'alpha': 2} train loss: 0.208249, valid loss:0.208026, loss_diff:-0.000223
{'alpha': 2} train loss: 0.208074, valid loss:0.208732, loss_diff:0.000658
{'alpha': 2} train loss: 0.208334, valid loss:0.207688, loss_diff:-0.000646
{'alpha': 2} train loss: 0.208270, valid loss:0.207946, loss_diff:-0.000324
{'alpha': 2} train loss: 0.208095, valid loss:0.208643, loss_diff:0.000548
=================>{'alpha': 2} loss:0.208207
{'alpha': 4} train loss: 0.208249, valid loss:0.208027, loss_diff:-0.000223
{'alpha': 4} train loss: 0.208074, valid loss:0.208731, loss_diff:0.000658
{'a

,param,val_loss_mean,val_loss_std
0,{'alpha': 1},0.208207,0.000409
1,{'alpha': 2},0.208207,0.000409
2,{'alpha': 4},0.208207,0.000409
3,{'alpha': 8},0.208207,0.000408


In [13]:
new_train[['f1', 'f2', 'f3']].corr()

,f1,f2,f3
f1,1.000000,0.998125,0.987907
f2,0.998125,1.000000,0.988439
f3,0.987907,0.988439,1.000000


In [14]:
default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

oof_val_pred, oof_test_pred, _ = get_oof_predictions( new_train, train_y, new_test, ml, 
                                                      default_params, seed=SEED, fit_params=fit_param, 
                                                      use_eval_set= False)

Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Fold 5 completed.


In [15]:
test_df = pd.read_csv("data/test.csv", usecols=['item_id'])
pd.DataFrame(np.clip(oof_test_pred,0,1), 
             index=test_df.item_id,
             columns=['deal_probability']).to_csv('stack_bagging_blending_411_complex.csv')

In [16]:
pd.DataFrame(np.clip(oof_val_pred,0,1), 
             columns=['deal_probability']).to_csv('stack_bagging_blending_411_complex_val.csv')

In [12]:
best_blend_test = None
best_score = None
min_w = 0.01
for a in tqdm(np.arange(min_w, 1+min_w-min_w*2, min_w)):
    for b in np.arange(min_w, 1-a+min_w-min_w*1, min_w):
        c = 1-a-b
        combined_res = a*results[0]['val_oof'] + \
                       b*results[1]['val_oof'] + \
                       c*results[2]['val_oof']

        score = clip_rmse(train_y, combined_res)
        if best_score is None or score < best_score:
            best_score = score
            print('best score updated: {:.6f}'.format(best_score), ' coefficient=> {}, {}, {}'.format(a, b, c))
            best_blend_test =  a*results[0]['test_oof'] + \
                               b*results[1]['test_oof'] + \
                               c*results[2]['test_oof']

  0%|                                                                                       | 0/98 [00:00<?, ?it/s]

best score updated: 0.209618  coefficient=> 0.01, 0.01, 0.98
best score updated: 0.209601  coefficient=> 0.01, 0.02, 0.97
best score updated: 0.209584  coefficient=> 0.01, 0.03, 0.96
best score updated: 0.209568  coefficient=> 0.01, 0.04, 0.95
best score updated: 0.209552  coefficient=> 0.01, 0.05, 0.94
best score updated: 0.209536  coefficient=> 0.01, 0.060000000000000005, 0.9299999999999999
best score updated: 0.209520  coefficient=> 0.01, 0.06999999999999999, 0.92
best score updated: 0.209504  coefficient=> 0.01, 0.08, 0.91
best score updated: 0.209489  coefficient=> 0.01, 0.09, 0.9
best score updated: 0.209473  coefficient=> 0.01, 0.09999999999999999, 0.89
best score updated: 0.209458  coefficient=> 0.01, 0.11, 0.88
best score updated: 0.209444  coefficient=> 0.01, 0.12, 0.87
best score updated: 0.209429  coefficient=> 0.01, 0.13, 0.86
best score updated: 0.209414  coefficient=> 0.01, 0.14, 0.85
best score updated: 0.209400  coefficient=> 0.01, 0.15000000000000002, 0.84
best score 

100%|██████████████████████████████████████████████████████████████████████████████| 98/98 [03:51<00:00,  2.36s/it]


In [ ]:
best_blend_test = None
best_score = None
min_w = 0.01
for a in np.arange(min_w, 1+min_w-min_w*4, min_w):
    for b in np.arange(min_w, 1-a+min_w-min_w*3, min_w):
        for c in np.arange(min_w, 1-a-b+min_w-min_w*2, min_w):
            for d in np.arange(min_w, 1-a-b-c+min_w-min_w*1, min_w):
                e = 1-a-b-c-d
                combined_res = a*results[0]['val_oof'] + \
                               b*results[1]['val_oof'] + \
                               c*results[2]['val_oof'] + \
                               d*results[3]['val_oof'] + \ 
                               e*results[4]['val_oof']
                
                score = clip_rmse(train_y, combined_res)
                if best_score is None or score < best_score:
                    best_score = score
                    print('best score updated:', best_score)
                    best_blend_test =  a*results[0]['test_oof'] + \
                                       b*results[1]['test_oof'] + \
                                       c*results[2]['test_oof'] + \
                                       d*results[3]['test_oof'] + \ 
                                       e*results[4]['test_oof']

In [8]:
test_df = pd.read_csv("data/test.csv", usecols=['item_id'])
pd.DataFrame(np.clip(best_blend_test,0,1), 
             index=test_df.item_id,
             columns=['deal_probability']).to_csv('stack_bagging_blend_no_xgb_meta.csv')